We are provided the train/test data in IOB format contained in a TSV file.

The train/test data in tsv format is passed into an annotation tool UBIAI which provides the output in same tsv format.

This tsv file then is then convertd to spaCy JSON format, which will be then converted to a .spacy binary file.

Note: Spacy takes input in .spacy format only

In [17]:
import spacy
import tensorflow

In [1]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2022-09-30 03:15:51--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-09-30 03:15:51--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?O-bBd9RCYpsy_oLde8sNnpFoBe-J_BzBi1PayrpQ1jn1QI64uuFcSMtlZM3hdMfGJoRIkie5zRN2MhDU26bjuSohnxHK6ctUW8gsaI2DneuGv

Download the files from Github

In [2]:
!git clone https://github.com/UBIAI/Fine_tune_BERT_with_spacy3.git

Cloning into 'Fine_tune_BERT_with_spacy3'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


Test/Train files are in tsv(IOB)... therefore we need convert them to Json file and then to .spacy binary file

In [3]:
!python -m spacy convert '/content/Fine_tune_BERT_with_spacy3/test.tsv' ./ -t json -n 1 -c iob
!python -m spacy convert "/content/Fine_tune_BERT_with_spacy3/train.tsv" ./ -t json -n 1 -c iob

2022-09-30 03:18:53.279612: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): test.json
2022-09-30 03:19:00.412704: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): train.json


Now we need to convert Json files to .spacy binary file

In [4]:
!python -m spacy convert "/content/test.json" ./ -t spacy
!python -m spacy convert "/content/train.json" ./ -t spacy

2022-09-30 03:19:06.114997: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Generated output file (11 documents): test.spacy
2022-09-30 03:19:11.512041: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Generated output file (77 documents): train.spacy


# Model Training:

In order to accelerate the training process, we need to run parallel processing on our GPU. To this end we install the NVIDIA 9.2 cuda library:

In [5]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb!dpkg -i cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb!apt-key add /var/cuda-repo-9–2-local/7fa2af80.pub!apt-get update!apt-get install cuda-9.2

--2022-09-30 03:19:13--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-09-30 03:19:13--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?6x5uDkJj_xKFT_-F8fAmtS8h4ukJ3gxCZD2IoJQjyoOyH4sawDpiwB7ubRueS6XXKfkYJ_wYwvmLyc_F2ymVXkMazf93ELY9YMQsMEMXvq6je

    Install the spacy library and spacy transformer pipeline:

In [6]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-09-30 03:19:51.985699: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 35 kB/s 
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 1.1 MB 10.5 MB/s 
     |████████████████████████████████| 4.7 MB 52.3 MB/s 
     |████████████████████████████████| 6.6 MB 41.0 MB/s 
     |████████████████████████████████| 163 kB 70.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


we install the pytorch machine learning library that is configured for cuda 9.2:

In [7]:
!pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 577.3 MB 3.0 kB/s 
     |████████████████████████████████| 12.5 MB 1.8 MB/s 
     |████████████████████████████████| 7.6 MB 4.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

After pytorch install, we need to install spacy transformers tuned for cuda 9.2 and change the CUDA_PATH and LD_LIBRARY_PATH as below. Finally, install the cupy library which is the equivalent of numpy library but for GPU:

In [8]:
!pip install -U spacy[cuda92,transformers]
!export CUDA_PATH=”/usr/local/cuda-9.2"
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH
!pip install cupy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55.0 MB 67.7 MB/s 
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
  ERROR: Failed cleaning build dir for cupy
Failed to build cupy
    Running setup.py install for cupy ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-u4xe0a9h/cupy_411d1065c1e949e5a339d9b1b1d9e999/setup.py'"'"'; __file__='"'"'/tmp/pip-install-u4xe0a9h/cupy_411d1065c1e949e5a339d9b1b1d9e999/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.ex

After updating the spacy config.cfg file with the training and test paths, we auto-fill the config file with the rest of the parameters that the BERT model will need

In [9]:
!python -m spacy init fill-config "/content/Fine_tune_BERT_with_spacy3/base_config.cfg" "/content/Fine_tune_BERT_with_spacy3/config_spacy.cfg"

2022-09-30 03:22:55.850852: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
/content/Fine_tune_BERT_with_spacy3/config_spacy.cfg
You can now add your data and train your pipeline:
python -m spacy train config_spacy.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Finally, we are ready to start the training:

In [10]:
!python -m spacy train --gpu-id -1 /content/Fine_tune_BERT_with_spacy3/config_spacy.cfg --output ./output

2022-09-30 03:23:02.567169: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-09-30 03:23:03,511] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-09-30 03:23:03,523] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-09-30 03:23:03,528] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-09-30 03:23:03,529] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you

At the end of the training, the model will be saved under folder model-best. The model scores are located in meta.json file inside the model-best folder

# Entity Extraction with Transformers

To test the model on a sample text, we need to load the model and run it on our text

In [18]:
nlp = spacy.load('/content/output/model-best')
text = ['''Qualifications- A thorough understanding of C# and .NET Core- Knowledge of good database design and usage- An understanding of NoSQL principles- Excellent problem solving and critical thinking skills- Curious about new technologies- Experience building cloud hosted, scalable web services- Azure experience is a plusRequirements- Bachelor's degree in Computer Science or related field(Equivalent experience can substitute for earned educational qualifications)- Minimum 4 years experience with C# and .NET- Minimum 4 years overall experience in developing commercial software''']
for doc in nlp.pipe(text, disable=["tagger", "parser"]):
  print([(ent.text, ent.label_) for ent in doc.ents])

[('Qualifications-', 'SKILLS'), ('A', 'SKILLS'), ('thorough', 'SKILLS'), ('understanding', 'SKILLS'), ('of', 'SKILLS'), ('C', 'SKILLS'), ('#', 'SKILLS'), ('and', 'SKILLS'), ('.NET', 'SKILLS'), ('Core-', 'SKILLS'), ('Knowledge', 'SKILLS'), ('of', 'SKILLS'), ('good', 'SKILLS'), ('database', 'SKILLS'), ('design', 'SKILLS'), ('and', 'SKILLS'), ('usage-', 'SKILLS'), ('An', 'SKILLS'), ('understanding', 'SKILLS'), ('of', 'SKILLS'), ('NoSQL', 'SKILLS'), ('principles-', 'SKILLS'), ('Excellent', 'SKILLS'), ('problem', 'SKILLS'), ('solving', 'SKILLS'), ('and', 'SKILLS'), ('critical', 'SKILLS'), ('thinking', 'SKILLS'), ('skills-', 'SKILLS'), ('Curious', 'SKILLS'), ('about', 'SKILLS'), ('new', 'SKILLS'), ('technologies-', 'SKILLS'), ('Experience', 'SKILLS'), ('building', 'SKILLS'), ('cloud', 'SKILLS'), ('hosted', 'SKILLS'), (',', 'SKILLS'), ('scalable', 'SKILLS'), ('web', 'SKILLS'), ('services-', 'SKILLS'), ('Azure', 'SKILLS'), ('experience', 'SKILLS'), ('is', 'SKILLS'), ('a', 'SKILLS'), ('plusRequ